In [2]:
try:
    import numpy as np
    import pandas as pd
    import re
    import tensorflow as tf
    from gensim.models import Word2Vec
    from nltk.tokenize import word_tokenize
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import Flatten
    from keras.layers.embeddings import Embedding
    from keras.preprocessing.text import one_hot
    from keras.preprocessing.sequence import pad_sequences
    from elasticsearch import Elasticsearch
    from elasticsearch import helpers 
    import json 
    from sklearn.cluster import KMeans
    from sklearn.metrics import mean_squared_error
    import warnings
    warnings.filterwarnings("ignore")
    print("Loaded.....")    
except Exception as e:
    print("Some modules are missing{}".format(e))

Loaded.....


In [3]:
movies = pd.read_csv("movies.csv")
ratings=pd.read_csv("ratings.csv")

In [4]:
def remove_stop_words(corpus):
    stop_words = ['is', 'a', 'will', 'be','of','to','for','an','are','am','the','and','if','in']
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stop_words:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))
    return results

In [5]:
titles=[]
for i in movies.title:
    
    i=re.sub(r'[^\w]', ' ', i)
    titles.append(i[:-6].lower())
titles=remove_stop_words(titles)
titles2=[]
for i in titles:
    titles2.append(i.split())

In [6]:
words=[]
for i in titles2:
    for k in i:
        words.append(k)

In [7]:
titles3=[]
for i in titles:
     titles3.append([i.strip()])

# word2vec

In [20]:
model =Word2Vec(titles3, min_count=1,size= 7,workers=3, window =2, sg = 1)

In [21]:
w2vtitles=[]
for i in titles3:
        w2vtitles.append(model[i].flatten())      

In [10]:
genres=movies.genres
vocab_length=25465

# One hot encoding

In [12]:
embedded_genres = [one_hot(sent, vocab_length) for sent in genres]
print(embedded_genres )

[[8082, 6947, 7782, 2689, 20475], [8082, 7782, 20475], [2689, 14442], [2689, 9798, 14442], [2689], [22769, 15981, 12341], [2689, 14442], [8082, 7782], [22769], [22769, 8082, 12341], [2689, 9798, 14442], [2689, 18632], [8082, 6947, 7782], [9798], [22769, 8082, 14442], [15981, 9798], [9798, 14442], [2689], [2689], [22769, 2689, 15981, 9798, 12341], [2689, 15981, 12341], [15981, 9798, 18632, 11446, 12341], [22769, 15981, 12341], [9798, 7606, 3634], [9798, 14442], [9798], [7782, 9798], [9798, 14442], [8082, 9798, 20475, 11446, 7606, 3634], [15981, 9798], [9798], [11446, 7606, 3634, 12341], [7782, 9798], [9798, 14442], [15981, 9798], [2268, 1623], [7782, 2689], [2689, 14442], [9798], [9798, 1996], [22769, 15981, 9798], [9798], [22769, 8082, 20475], [2689, 9798, 12341], [9798, 14442], [11446, 12341], [6947, 7782, 9798, 6111, 14442], [9798, 14442], [15981, 11446, 12341], [2689, 9798, 14442], [8082, 9798], [7782, 2689], [9798], [9798], [2689, 9798, 14442], [9798, 11446], [8082, 7782, 20475], [

In [22]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(genres, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

# Data for keras

In [23]:
padded_genres = pad_sequences(embedded_genres, length_long_sentence, padding='post')
print(padded_genres)


[[ 8082  6947  7782  2689 20475]
 [ 8082  7782 20475     0     0]
 [ 2689 14442     0     0     0]
 ...
 [ 2268     0     0     0     0]
 [ 2689     0     0     0     0]
 [ 2268     0     0     0     0]]


In [27]:
which_user=int(input("Tell me your user Id"))
m=ratings[(ratings["userId"]==which_user)]
m.rename(columns={'rating': 'user_rating'}, inplace=True)
m.drop("timestamp",axis=1,inplace=True)
m.drop("userId",axis=1,inplace=True)

n=pd.merge(movies,m,on="movieId",how="outer")
yrating=n.user_rating.fillna(0)

Tell me your user Id13


In [28]:
data=[]
for row in enumerate(w2vtitles):
    data.append(np.concatenate((row[1],padded_genres[row[0]]),axis=0).tolist())
    

In [29]:
x=[]
for index,rating in enumerate(yrating):
    if rating>0:
        x.append(data[index])
X = np.array(x)

In [30]:
y=[]
for i in m.user_rating:
    y.append([i])    
Y = np.array(y)



# model training

In [31]:
model = Sequential()
model.add(Embedding(vocab_length, 40, input_length=12))
model.add(Flatten())
model.add(Dense(1, activation='selu'))

In [79]:
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 12, 40)            1018600   
_________________________________________________________________
flatten_1 (Flatten)          (None, 480)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 481       
Total params: 1,019,081
Trainable params: 1,019,081
Non-trainable params: 0
_________________________________________________________________
None


In [80]:
model.fit(x,y, epochs=150, verbose=1)

Epoch 1/150
2/2 [==============================] - 0s 11ms/step - loss: 14.4201 - root_mean_squared_error: 3.7974
Epoch 2/150
2/2 [==============================] - 0s 11ms/step - loss: 14.0469 - root_mean_squared_error: 3.7479
Epoch 3/150
2/2 [==============================] - 0s 11ms/step - loss: 13.7198 - root_mean_squared_error: 3.7040
Epoch 4/150
2/2 [==============================] - 0s 9ms/step - loss: 13.3924 - root_mean_squared_error: 3.6596
Epoch 5/150
2/2 [==============================] - 0s 12ms/step - loss: 13.0538 - root_mean_squared_error: 3.6130
Epoch 6/150
2/2 [==============================] - 0s 11ms/step - loss: 12.7151 - root_mean_squared_error: 3.5658
Epoch 7/150
2/2 [==============================] - 0s 11ms/step - loss: 12.3691 - root_mean_squared_error: 3.5170
Epoch 8/150
2/2 [==============================] - 0s 12ms/step - loss: 12.0177 - root_mean_squared_error: 3.4667
Epoch 9/150
2/2 [==============================] - 0s 14ms/step - loss: 11.6538 - root_me

2/2 [==============================] - 0s 9ms/step - loss: 0.1913 - root_mean_squared_error: 0.4374
Epoch 144/150
2/2 [==============================] - 0s 10ms/step - loss: 0.1906 - root_mean_squared_error: 0.4365
Epoch 145/150
2/2 [==============================] - 0s 11ms/step - loss: 0.1898 - root_mean_squared_error: 0.4356
Epoch 146/150
2/2 [==============================] - 0s 11ms/step - loss: 0.1887 - root_mean_squared_error: 0.4344
Epoch 147/150
2/2 [==============================] - 0s 12ms/step - loss: 0.1879 - root_mean_squared_error: 0.4335
Epoch 148/150
2/2 [==============================] - 0s 14ms/step - loss: 0.1871 - root_mean_squared_error: 0.4326
Epoch 149/150
2/2 [==============================] - 0s 11ms/step - loss: 0.1862 - root_mean_squared_error: 0.4315
Epoch 150/150
2/2 [==============================] - 0s 9ms/step - loss: 0.1854 - root_mean_squared_error: 0.4306


In [81]:
x_new=[]
for index,rating in enumerate(yrating):
    if rating==0:
        x_new.append(data[index])
X_new = np.array(x_new)

In [19]:
Y_new=model.predict(X_new)
for i in Y_new:
    print(i)

AttributeError: 'Word2Vec' object has no attribute 'predict'

In [83]:
predictions = pd.DataFrame(yrating) 

In [84]:
i=0
for index,row in predictions.iterrows():
    if row[0]==0:
            predictions.iloc[index,0]=Y_new[i] 
            i=i+1         

In [85]:
predictions=predictions.rename(columns={'user_rating':'prediction_rating'})

In [86]:
predictions
m=pd.merge(movies, predictions, left_index=True, right_index=True)
m

,movieId,title,genres,prediction_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.000000
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.993098
2,3,Grumpier Old Men (1995),Comedy|Romance,3.404216
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3.919714
4,5,Father of the Bride Part II (1995),Comedy,3.284654
...,...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,4.245365
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,3.387571
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,3.225597
9123,164977,The Gay Desperado (1936),Comedy,3.284654


In [87]:
userinfo=ratings[(ratings["userId"]==which_user)]
n=pd.merge(m,userinfo,on="movieId",how="outer")
n.drop("timestamp",axis=1,inplace=True)
n.drop("userId",axis=1,inplace=True)
n=n.fillna(0)

In [88]:
ratings1=pd.read_csv("ratings.csv")
avgratings=ratings1.groupby(['movieId'])['rating'].mean()
avgratings

movieId
1         3.872470
2         3.401869
3         3.161017
4         2.384615
5         3.267857
            ...   
161944    5.000000
162376    4.500000
162542    5.000000
162672    3.000000
163949    5.000000
Name: rating, Length: 9066, dtype: float64

In [89]:
m=pd.merge(n,avgratings,on="movieId",how="outer")
m=m.rename(columns={'rating_x':'user_rating','rating_y':'mean_rating'})

# Clustering from task3

In [90]:
movies3=movies.join(movies.pop('genres').str.get_dummies("|"))
movies3.replace(0, np.nan,inplace=True)

In [91]:
df=pd.merge(movies3,ratings,on="movieId", how="outer")

In [92]:
genres=movies3.columns[2:]

In [93]:
def f(x):
    y = x.dropna()
    return np.nan if y.empty else y

means_list=[]
genre_rating=pd.DataFrame()
for i in genres:
    
    tempdf=df.groupby(['userId',i] )['rating'].mean().agg(f).to_frame(name =i+" average")
    means_list.append(tempdf)
    means=means_list[0]
for i in range(1,len(means_list)):
    means=pd.merge(means,means_list[i],on="userId", how="outer")
means=means.sort_values(by=['userId'])
means.drop('(no genres listed) average',axis='columns', inplace=True)

In [33]:
data4kmeans=means.fillna(0)

In [94]:
X=data4kmeans[['Action average','Adventure average','Animation average','Children average','Comedy average','Crime average','Documentary average','Drama average','Fantasy average','Film-Noir average','Horror average','IMAX average','Musical average','Mystery average','Romance average','Sci-Fi average','Thriller average','War average','Western average']].values

In [95]:
kmeans = KMeans(n_clusters=8)
predictions = kmeans.fit_predict(X)

In [96]:
n_clusters=8
clusters = [X[predictions== i] for i in range(n_clusters)]

centroids = kmeans.cluster_centers_
ClusterMeans=pd.DataFrame(columns=['userId','Action average','Adventure average','Animation average','Children average','Comedy average','Crime average','Documentary average','Drama average','Fantasy average','Film-Noir average','Horror average','IMAX average','Musical average','Mystery average','Romance average','Sci-Fi average','Thriller average','War average','Western average'])
myUserId=1
for i in predictions:
    cluster_number=i
    myValues=centroids[cluster_number]
    new_row = {'userId':myUserId,'Action average':myValues[0],'Adventure average':myValues[1],'Animation average':myValues[2],'Children average':myValues[3],'Comedy average':myValues[4],'Crime average':myValues[5],'Documentary average':myValues[6],'Drama average':myValues[7],'Fantasy average':myValues[8],'Film-Noir average':myValues[9],'Horror average':myValues[10],'IMAX average':myValues[11],'Musical average':myValues[12],'Mystery average':myValues[13],'Romance average':myValues[14],'Sci-Fi average':myValues[15],'Thriller average':myValues[16],'War average':myValues[17],'Western average':myValues[18]}
    ClusterMeans = ClusterMeans.append(new_row, ignore_index=True)
    myUserId+=1
ClusterMeans.set_index('userId', inplace=True)

In [97]:
filleddata=data4kmeans.where(data4kmeans!=0,ClusterMeans)

In [98]:
m=pd.merge(m,movies3,on="movieId",how="outer")
m=m.drop(columns="title_y")
m=m.drop(columns="(no genres listed)")

In [99]:
m=m.rename(columns={'Action':'Action average','Adventure':'Adventure average','Animation':'Animation average','Children':'Children average','Comedy':'Comedy average','Crime':'Crime average','Documentary':'Documentary average','Drama':'Drama average','Fantasy':'Fantasy average','Film-Noir':'Film-Noir average','Horror':'Horror average','IMAX':'IMAX average','Musical':'Musical average','Mystery':'Mystery average','Romance':'Romance average','Sci-Fi':'Sci-Fi average','Thriller':'Thriller average','War':'War average','Western':'Western average'})

In [100]:
r=filleddata.loc[which_user,:]
m=m.where(m!=1.0,r ,axis=1)
m.iloc[0, m.columns.get_loc('movieId')] = 1.0
dfend=m
dfend

,movieId,title_x,genres,prediction_rating,user_rating,mean_rating,Action average,Adventure average,Animation average,Children average,...,Film-Noir average,Horror average,IMAX average,Musical average,Mystery average,Romance average,Sci-Fi average,Thriller average,War average,Western average
0,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.000000,5.0,3.872470,NaN,3.85,4.0,3.884615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Jumanji (1995),Adventure|Children|Fantasy,3.993098,0.0,3.401869,NaN,3.85,NaN,3.884615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Grumpier Old Men (1995),Comedy|Romance,3.404216,0.0,3.161017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.9,NaN,NaN,NaN,NaN
3,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,3.919714,0.0,2.384615,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.9,NaN,NaN,NaN,NaN
4,5.0,Father of the Bride Part II (1995),Comedy,3.284654,0.0,3.267857,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,162672.0,Mohenjo Daro (2016),Adventure|Drama|Romance,4.245365,0.0,3.000000,NaN,3.85,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.9,NaN,NaN,NaN,NaN
9121,163056.0,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,3.387571,0.0,NaN,3.75,3.85,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.583333,NaN,NaN,NaN
9122,163949.0,The Beatles: Eight Days a Week - The Touring Y...,Documentary,3.225597,0.0,5.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9123,164977.0,The Gay Desperado (1936),Comedy,3.284654,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
dfend.replace(0,np.nan,inplace=True)
dfend["cluster_ratings_prediction"]=dfend[['Action average','Adventure average','Animation average','Children average','Comedy average','Crime average','Documentary average','Drama average','Fantasy average','Film-Noir average','Horror average','IMAX average','Musical average','Mystery average','Romance average','Sci-Fi average','Thriller average','War average','Western average']].mean(axis=1)

dfend=dfend.fillna(0)
dfend

,movieId,title_x,genres,prediction_rating,user_rating,mean_rating,Action average,Adventure average,Animation average,Children average,...,Horror average,IMAX average,Musical average,Mystery average,Romance average,Sci-Fi average,Thriller average,War average,Western average,cluster_ratings_prediction
0,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.000000,5.0,3.872470,0.00,3.85,4.0,3.884615,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,3.824196
1,2.0,Jumanji (1995),Adventure|Children|Fantasy,3.993098,0.0,3.401869,0.00,3.85,0.0,3.884615,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,3.850932
2,3.0,Grumpier Old Men (1995),Comedy|Romance,3.404216,0.0,3.161017,0.00,0.00,0.0,0.000000,...,0.0,0.0,0.0,0.0,3.9,0.000000,0.0,0.0,0.0,3.734091
3,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,3.919714,0.0,2.384615,0.00,0.00,0.0,0.000000,...,0.0,0.0,0.0,0.0,3.9,0.000000,0.0,0.0,0.0,3.810823
4,5.0,Father of the Bride Part II (1995),Comedy,3.284654,0.0,3.267857,0.00,0.00,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,3.568182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,162672.0,Mohenjo Daro (2016),Adventure|Drama|Romance,4.245365,0.0,3.000000,0.00,3.85,0.0,0.000000,...,0.0,0.0,0.0,0.0,3.9,0.000000,0.0,0.0,0.0,3.904762
9121,163056.0,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,3.387571,0.0,0.000000,3.75,3.85,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,3.583333,0.0,0.0,0.0,3.750379
9122,163949.0,The Beatles: Eight Days a Week - The Touring Y...,Documentary,3.225597,0.0,5.000000,0.00,0.00,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.024793
9123,164977.0,The Gay Desperado (1936),Comedy,3.284654,0.0,0.000000,0.00,0.00,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,3.568182


In [102]:
ENDPOINT= "http://localhost:9200/"
es=Elasticsearch(timeout=600,hosts=ENDPOINT)

In [104]:
data4m=dfend.to_dict("records")

def generator(data4m):
    for c,line in enumerate(data4m):
        yield{
         '_index':'data4m',
         '_type' :'_doc',
         '_id':line.get('movieId'),
         '_source':{
             'title':line.get('title_x',["No Data"]),
             'genres':line.get('genres',["No Data"]),
             'rating':line.get('mean_rating',["No Data"]),
             'user_rating':line.get('user_rating',["No Data"]),
             'Action average':line.get('Action average',["No Data"]),
             'Adventure average':line.get('Adventure average',["No Data"]),
             'Animation average':line.get('Animation average',["No Data"]),
             'Children average':line.get('Children average',["No Data"]),
             'Comedy average':line.get('Comedy average',["No Data"]),
             'Crime average ':line.get('Crime average',["No Data"]),
             'Documentary average':line.get('Documentary average',["No Data"]),
             'Drama average':line.get('Drama average',["No Data"]),
             'Fantasy average':line.get('Fantasy average',["No Data"]),
             'Film-Noir average':line.get('Film-Noir average',["No Data"]),
             'Horror average':line.get('Horror average',["No Data"]),
             'IMAX average':line.get('IMAX average',["No Data"]),
             'Musical average':line.get('Musical average',["No Data"]),
             'Mystery average':line.get('Mystery average',["No Data"]),
             'Romance average':line.get('Romance average',["No Data"]),
             'Sci-Fi average':line.get('Sci-Fi average',["No Data"]),
             'Thriller average':line.get('Thriller average',["No Data"]),
             'War average':line.get('War average',["No Data"]),
             'Western average':line.get('Western average',["No Data"]),
             'predictionrating':line.get('prediction_rating',["No Data"]),
             'clusterprediction':line.get('cluster_ratings_prediction',["No Data"])
         }
             } 
      
    gen=generator(data4m)

In [105]:
 settings = {
    'settings': {
            'number_of_shards': 1,
            'number_of_replicas': 1,
            "similarity": {
             "custom_bm25": {
                "type": "BM25",
                "b": 0,
                "k1" : 0.9
                 
             }
          }
                
            }
    ,
    
    'mappings': {
        
            "properties": {
                "title": {
                    "type": "text",
                    "analyzer": "english"
                },
                "genres":{
                    "type": 'text',
                    "analyzer":"english"
                },
            
             "rating":{
                    "type": "short"
                
                    
             },
                          
          
             "user_rating":{
                    "type": "short"
                    
               }, 
            "ratingprediction":{
                "type":"short"
            },
            "clusterprediction":{
                "type":"short"
            }    
            }
        
        
        
           }
    
} 

In [106]:
my=es.indices.create(index="dfend",ignore=[400,404],body=settings)

# INSERT TO ES

In [107]:
try:
    res=helpers.bulk(es,generator(data4m))
    print("Working")
except Exception as e:
    print("Done")

Working


In [108]:
user_request=input("What are you looking for?")
query_body = {
  "query": {
      "multi_match":{
          "query":user_request,
          "fields":["title" ,"genres"]
      }
  }
}

      
# Pass the query dictionary to the 'body' parameter of the
# client's Search() method, and have it return results:
result = es.search(index="data4m", body=query_body,size=10000,)
    

What are you looking for?batman


# NEW METRIC 

In [48]:
def metric(BM25,rating,user_rating,ratingprediction,cluster_rating):
        if (user_rating>0 and rating >0 ) :         
            return (0.2*(BM25+1*rating+1*user_rating +1*ratingprediction+1*cluster_rating))
        elif (user_rating==0 or rating ==0 ): 
            return ((1/4))*(BM25+1*rating+1*user_rating+1*ratingprediction+1*cluster_rating)
        else :
            return BM25

In [49]:
for i in result["hits"]["hits"]:
    
    BM25_score=i["_score"]*0.5
    rating=i["_source"]["rating"]
    user_rating=i["_source"]["user_rating"]
    prediction_rating=i["_source"]["predictionrating"]
    cluster_rating=i["_source"]["clusterprediction"]
    new_metric=metric(BM25_score,rating,user_rating,prediction_rating,cluster_rating)
    i['new_score']=new_metric

result["hits"]["hits"]=sorted(result["hits"]["hits"],key=lambda i:i['new_score'],reverse=True )
    
print ("total hits:", len(result["hits"]["hits"]))

for i in result['hits']['hits']:
    print(i['_source']['title']+" "+i['_source']['genres']+"\n")

total hits: 12
Batman Begins (2005) Action|Crime|IMAX

Batman (1966) Action|Adventure|Comedy

Batman (1989) Action|Crime|Thriller

Batman: Mask of the Phantasm (1993) Animation|Children

Batman Returns (1992) Action|Crime

Batman: The Dark Knight Returns, Part 1 (2012) Action|Animation|Sci-Fi

Batman: The Dark Knight Returns, Part 2 (2013) Action|Animation

Batman Forever (1995) Action|Adventure|Comedy|Crime

Batman: Under the Red Hood (2010) Action|Animation

Batman: Year One (2011) Action|Animation|Crime

Batman & Robin (1997) Action|Adventure|Fantasy|Thriller

Batman v Superman: Dawn of Justice (2016) Action|Adventure|Fantasy|Sci-Fi

